# K分割交差検証

#### 概略図

![](１.png)

トレーニングデータセットをランダムにk個に分割　<p>
そのうちk-1個をモデルのトレーニングに使用<p>
1個をテスト用に使用<p>
<p>
この手順をk回行うことで　k個のモデルと性能評価を取得できる<p>

##### 一般的に、k分割交差検証は、モデルのチューニングに使用される。
満足のいく汎化性能が得られる最適なハイパーパラメータ値を見つけ出すために使用される<p>


腫瘍データ

In [5]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases'
                     '/breast-cancer-wisconsin/wdbc.data', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


M→1 B→0 <p>
トレーニング：テスト　＝ 8 : 2

In [6]:
from sklearn.preprocessing import LabelEncoder

X = df.loc[:, 2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.transform(['M', 'B'])

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.20, random_state=1)

C:\Users\admin\python\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## パイプラインを用いて、変換器と推定器を結合

変換器：データの標準化　＆　PCA　<p>
推定器：ロジスティック回帰

データ標準化　→　30次元から2次元に圧縮する主成分分析(PCA) <p>
ロジスティック回帰で分類

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression


#### パイプライン　オブジェクト

In [8]:
from sklearn.pipeline import Pipeline

これを利用して、各要素にアクセスできる

In [9]:
pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', LogisticRegression(random_state=1))])


![](ダウンロード.png)

### K分割交差検証

In [10]:
import numpy as np

# K分割交差検証のライブラリを読み込む
from sklearn.cross_validation import StratifiedKFold
    
# K分割交差検証のインスタンス生成
# y →　分割元データ(トレーニングデータセット)
# n_folds →　分割数
kfold = StratifiedKFold(y=y_train,
                        n_folds=10,
                        random_state=1)

scores = []

# データをモデルに適合
# テストデータの正解率を計算　pipe_lr.score
# リストに正解率を追加

for k, (train, test) in enumerate(kfold):
    
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)

# kは0からスタートで、左から
# 分割番号　, 0以上の要素数　,　正解率
    print('Fold: %s, Class dist.: %s, Acc: %.3f' % 
          (k+1, np.bincount(y_train[train]), score))
# 正解率の平均と標準偏差
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

Fold: 1, Class dist.: [256 153], Acc: 0.891
Fold: 2, Class dist.: [256 153], Acc: 0.978
Fold: 3, Class dist.: [256 153], Acc: 0.978
Fold: 4, Class dist.: [256 153], Acc: 0.913
Fold: 5, Class dist.: [256 153], Acc: 0.935
Fold: 6, Class dist.: [257 153], Acc: 0.978
Fold: 7, Class dist.: [257 153], Acc: 0.933
Fold: 8, Class dist.: [257 153], Acc: 0.956
Fold: 9, Class dist.: [257 153], Acc: 0.978
Fold: 10, Class dist.: [257 153], Acc: 0.956

CV accuracy: 0.950 +/- 0.029


#### scikit-learnにはk分割交差検証の性能指標を簡単に算出する関数が用意されている。
以下に示す。

In [11]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [ 0.89130435  0.97826087  0.97826087  0.91304348  0.93478261  0.97777778
  0.93333333  0.95555556  0.97777778  0.95555556]
CV accuracy: 0.950 +/- 0.029
